**Task 1 : fetch the transcripts of the competitors**
- using beautifulSoup we are tyring to webscrape the motely-fool site and get the transcripts data.

# Task 1 : Fetch Transcripts of Competitors

- We are using beautifulSoup to scrape Transcript from Website

# Task 2 : Summarize the Transcripts

1.) SUMMARY
- Key Financial Metrics like Loan Growth, Credit Quality, P & L Details, Deposits, Profitability
- Management Commentary Highlights - their focus & strategic framework
- Insights into Business Verticals
- Forward Guidance & Outlook

2.) STRATEGIC UPDATES
- Strategic Updates like Loan Growth, Credit Quality, P & L Details, Deposits, Profitability etc

3.) GUIDANCE OUTLOOK
- Forward Earnings Guidance
- Outlook for Future Business Performance
- Expected Earnings Growth or Decline
- Reasons behind Growth or Decline in Earnings

4.) RISK ANALYSIS
- Risks discussed in Transcript
- How management is responding to Risks

5.) Q & A SUMMARY
- Summary of Top Questions & Answers in short form of most important Q&A discussed in Transcript
- Some Examples of Questions related to Loan Growth, Credit Quality, Growth in Digital Offerings, Portfolio Information, Outlook on Credit Cost etc.

**Tafunctionality**


* Keyword Definition: The code defines a comprehensive list of keywords related to financial topics that will be used to identify relevant sections in a transcript.

* Embedding Generation: It utilizes a pre-trained SentenceTransformer model (all-MiniLM-L6-v2) to create vector embeddings for both the keywords and the sentences extracted from the transcript.

* Cosine Similarity Calculation: For each sentence in the transcript, it calculates the cosine similarity between the sentence embeddings and the keyword embeddings to identify sentences that are relevant based on a specified similarity threshold.

* Output Formatting: It compiles the relevant sentences into a summary and outputs them in a bullet-point format, or indicates if no relevant content was found.



In [2]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 14.8 MB/s eta 0:00:00


In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import spacy
from sentence_transformers import SentenceTransformer, util


class Stock:
    def __init__(self, symbol, name, exchange=None, sector=None, industry=None):
        self.symbol = symbol
        self.name = name
        self.exchange = exchange
        self.sector = sector
        self.industry = industry
        self.transcripts = []

    def __str__(self):
        parts = [f"{self.symbol}: {self.name}"]
        if self.exchange:
            parts.append(f"({self.exchange})")
        if self.sector:
            parts.append(f"Sector: {self.sector}")
        if self.industry:
            parts.append(f"Industry: {self.industry}")
        return " | ".join(parts)


class Transcript:
    def __init__(self, stock, year, quarter, content):
        self.stock = stock
        self.year = year
        self.quarter = quarter
        self.content = content
        self.summary = None

    def __str__(self):
        return f"{self.stock.symbol} - {self.year} Q{self.quarter} Earnings Call Transcript"


def fetch_transcript(stock, year, quarter):
    url = f"https://www.earningscall.ai/stock/transcript/{stock.symbol}-{year}-Q{quarter}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')


        transcript_element = soup.find('div', class_='whitespace-pre-line')

        if transcript_element:
            content = transcript_element.get_text(strip=True)
            return Transcript(stock, year, quarter, content)
        else:
            print(f"Couldn't find transcript content for {stock.symbol} - {year} Q{quarter}\n")
            return None
    except requests.RequestException as e:
        print(f"Error fetching transcript for {stock.symbol} - {year} Q{quarter}: {e}")
        return None


def fetch_all_quarterly_transcripts(stock, year):
    for quarter in range(1, 5):
        transcript = fetch_transcript(stock, year, quarter)
        if transcript:
            stock.transcripts.append(transcript)
            print(f"Fetched transcript for {transcript}\n")
        else:
            print(f"Failed to fetch transcript for {stock.symbol} - {year} Q{quarter}\n")
        time.sleep(1)


def summarize_transcripts(transcripts, stock, year):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    keywords_summary = [
        "Wealth Management", "Acquisition", "Advisers", "client acquisition",
        "sweep account", "client assets", "client net flows", "nii",
        "retirement and protection", "fa recruiting", "advisor", "cash balance",
        "sweep money market", "aum", "cd", "awm", "cash sweep", "deposits",
        "loans", "wealth manager", "crm", "client flows", "client flow rates",
        "sweep balances", "general and administrative expenses", "wrap account",
        "brokerage account", "portfolios", "wrap flows", "acquisitions",
        "acquisition", "integration", "merger", "strategy", "portfolio", "advisors",
        # Additional keywords for the summary
        "Financial Performance", "Loan Growth", "Credit Quality", "Profit & Loss (P&L)",
        "Revenue Growth", "Deposit Trends", "Net Interest Margin", "Asset Under Management (AUM)",
        "Return on Equity (ROE)", "Operating Efficiency", "Profitability Ratios",
        "Management Commentary", "Strategic Focus", "Competitive Positioning",
        "Business Segments/Verticals", "Expense Management", "Cost Control",
        "Capital Allocation", "Technology Investments", "ESG (Environmental, Social, Governance)",
        "Shareholder Returns"
    ]

    keyword_embeddings = model.encode(keywords_summary, convert_to_tensor=True)
    nlp = spacy.load("en_core_web_sm")

    qtr = 1;
    for transcript in transcripts:
        print(f"\nSummarizing transcript for {stock.symbol}-{year} of Q{qtr}")
        transcript_sentences = [sent.text for sent in nlp(transcript.content).sents]
        transcript_embeddings = model.encode(transcript_sentences, convert_to_tensor=True)

        threshold = 0.5
        relevant_sections = {
            "SUMMARY": []
        }

        for i, transcript_embedding in enumerate(transcript_embeddings):
            cos_similarities = util.pytorch_cos_sim(transcript_embedding, keyword_embeddings)[0]
            if any(similarity > threshold for similarity in cos_similarities):
                relevant_sections["SUMMARY"].append(transcript_sentences[i])

        transcript.summary = relevant_sections["SUMMARY"]
        file_name = f"{stock.symbol}_{year}_Q{qtr}_transcript_summary.txt"
        !touch file_name
        transcript_summary = s = ''.join(transcript.summary)
        with open('/content/' + file_name, 'w') as writefile:
            for sentence in transcript.summary:
              writefile.write(f"    - {sentence.strip()}\n")

        print(f"\nSummarized transcript for {stock.symbol}-{year} of Q{qtr} Successfully!")
        qtr = qtr + 1


stocks = [
    Stock("JPM", "JPMorgan Chase & Co.", "NYSE", "Financial Services", "Banks—Diversified"),
    # Stock("MS", "Morgan Stanley", "NYSE", "Financial Services", "Capital Markets"),
    # Stock("JPM", "JPMorgan Chase & Co.", "NYSE", "Financial Services", "Banks—Diversified"),
    # Stock("BAC", "Bank of America Corporation", "NYSE", "Financial Services", "Banks—Diversified"),
    # Stock("SF", "Stifel Financial Corp.", "NYSE", "Financial Services", "Capital Markets"),
    # Stock("RJF", "Raymond James Financial Inc.", "NYSE", "Financial Services", "Capital Markets"),
    # Stock("AMP", "Ameriprise Financial Inc.", "NYSE", "Financial Services", "Asset Management"),
    # Stock("LPL", "LPL Financial Holdings Inc.", "NASDAQ", "Financial Services", "Capital Markets"),
    # Stock("UBS", "UBS Group AG", "NYSE", "Financial Services", "Banks—Diversified"),
    # Stock("SCHW", "The Charles Schwab Corporation", "NYSE", "Financial Services", "Capital Markets"),
    # Stock("WFC", "Wells Fargo & Company", "NYSE", "Financial Services", "Banks—Diversified"),
]


current_year = datetime.now().year

for stock in stocks:
    print(f"\nFetching transcripts for {stock.symbol}:\n")
    fetch_all_quarterly_transcripts(stock, current_year)


for stock in stocks:
    if stock.transcripts:
        print(f"\nSummarizing transcripts for {stock.symbol}:\n")
        summarize_transcripts(stock.transcripts, stock, current_year)


for stock in stocks:
    print(f"\n{stock.name} ({stock.symbol}) Transcripts Summary:")
    for transcript in stock.transcripts:

        print(f"  - {transcript} Summary:")

        if transcript.summary:
            for sentence in transcript.summary:
                print(f"    - {sentence.strip()}")
            print(f"\n\n")
        else:
            print("No summary found!")



Fetching transcripts for JPM:

Fetched transcript for JPM - 2024 Q1 Earnings Call Transcript

Fetched transcript for JPM - 2024 Q2 Earnings Call Transcript

Fetched transcript for JPM - 2024 Q3 Earnings Call Transcript

Couldn't find transcript content for JPM - 2024 Q4

Failed to fetch transcript for JPM - 2024 Q4


Summarizing transcripts for JPM:



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Summarizing transcript for JPM-2024 of Q1

Summarized transcript for JPM-2024 of Q1 Successfully!

Summarizing transcript for JPM-2024 of Q2

Summarized transcript for JPM-2024 of Q2 Successfully!

Summarizing transcript for JPM-2024 of Q3

Summarized transcript for JPM-2024 of Q3 Successfully!

JPMorgan Chase & Co. (JPM) Transcripts Summary:
  - JPM - 2024 Q1 Earnings Call Transcript Summary:
    - The firm reported net income of $13.4 billion, EPS of $4.44 on revenue of $42.5 billion and delivered an ROTCE of 21%.
    - And we have seen strong net inflows across AWM as well as in the CCB and Wealth Management business.
    - Revenue of $40.9 billion was up $1.5 billion or 4% year-on-year.
    - And Markets revenue was down $400 million or 5% year-on-year.
    - Expenses of $22 billion were up $1.8 billion or 9% year-on-year driven by higher compensation, including growth in employees and the increase to the FDIC special assessment.
    - On to balance sheet and capital on Page 3.
  

# Task 3 : Send Email

- Automated Mail should be sent to list of stakeholders once the PPTs are generated


In [15]:
import os
import zipfile
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

# Function to create a zip file
def create_zip_file(file_paths, zip_name):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for file in file_paths:
            zipf.write(file, os.path.basename(file))

# List of file paths (replace with your actual file paths)
file_paths = [
    # "/content/JPM_2024_Q1_transcript_summary.txt"
    # "/content/JPM_2024_Q2_transcript_summary.txt"
    # ... add all 10 file paths here
]

# Function to send email
def send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path):
    # Create a multipart message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    for stock in stocks:
      year = datetime.now().year
      for qtr in range(1, 5):
        path = "/content/" + f"{stock.symbol}_{year}_Q{qtr}_transcript_summary.txt"
        if os.path.isfile(path):
          file_paths.append(path)

    print(file_paths)

    zip_name = f"FY_{year}_Competitors_Transcript_Summary.zip"

    # Create ZIP file
    create_zip_file(file_paths, zip_name)

    # Open ZIP file in binary mode
    with open(attachment_path, "rb") as attachment:
        # Add file as application/octet-stream
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {os.path.basename(attachment_path)}",
    )

    # Add attachment to message
    message.attach(part)

    # Convert message to string
    text = message.as_string()

    # Log in to server using secure context and send email
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, text)



# Email details
sender_email = "bhagyarana1002@gmail.com"
sender_password = "twnw yalw pfxx xwvp"
stakeholders = [
    "bhagyarana2001@gmail.com",
    # Add more stakeholder emails as needed
]
year = datetime.now().year
zip_name = f"FY_{year}_Competitors_Transcript_Summary.zip"
subject = f"Wealth Management Competitor Earnings Call Summaries - {year}"
body = f"""Dear Stakeholder,

Please find attached a zip file containing summaries of the FY-{year} earnings call transcripts for our top 10 competitors in wealth management.
If you have any questions or need further information, please don't hesitate to reach out.

Best regards,
FinSolve-Team 10
"""

# Send email to each stakeholder
for stakeholder_email in stakeholders:
    try:
        send_email(sender_email, sender_password, stakeholder_email, subject, body, zip_name)
        print(f"Email sent successfully to {stakeholder_email}")
    except Exception as e:
        print(f"Failed to send email to {stakeholder_email}. Error: {str(e)}")

print("Email sending process completed.")

['/content/JPM_2024_Q1_transcript_summary.txt', '/content/JPM_2024_Q2_transcript_summary.txt', '/content/JPM_2024_Q3_transcript_summary.txt']
Email sent successfully to bhagyarana2001@gmail.com
Email sending process completed.
